In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
# Example with tf.train.batch dynamic padding
# ==================================================

tf.reset_default_graph()

# Create a tensor [0, 1, 2, 3, 4 ,...]
x = tf.range(1, 51, name="x")

# A queue that outputs 0,1,2,3,...
range_q = tf.train.range_input_producer(limit=51, shuffle=True)
slice_end = range_q.dequeue()

# Slice x to variable length, i.e. [0], [0, 1], [0, 1, 2], ....
y = tf.slice(x, [0], [slice_end], name="y")

print(y)

# Batch the variable length tensor with dynamic padding
batched_data, batch_leng = tf.train.batch(
    tensors=[y, slice_end],
    batch_size=100,
    dynamic_pad=True,
    name="y_batch"
)

print(batched_data)
print(batch_leng) 

# Run the graph
# tf.contrib.learn takes care of starting the queues for us
res = tf.contrib.learn.run_n({"y": batched_data, 'len':batch_leng}, n=100, feed_dict=None)

# Print the result
print len(res)
print res[0]["y"]
print res[0]["len"]

input_label= np.array([batch_input["y"] for batch_input in res])
input_length= np.array([batch_input["len"] for batch_input in res])
y_label = (np.array([batch_input["y"] for batch_input in res])-1) * (1*(input_label>0))
input_mask = 1.0*(input_label>0)

print input_label.shape
print input_mask.shape
print input_length.shape
print y_label.shape

In [ ]:
from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.python.ops import tensor_array_ops
# Parameters
learning_rate = 0.0001
training_iters = 100000
batch_size = 100
display_step = 10
epoch_per_step = 100

# Network Parameters
n_input = 1
n_steps = 50 # timesteps
n_hidden = 256 # hidden layer num of features
n_classes = 50 # total classes

# tf Graph input
x = tf.placeholder("float32", [None, n_steps, n_input])
y = tf.placeholder("int32", [None, n_steps])
mask = tf.placeholder("float32", [None, n_steps])
len_info = tf.placeholder("int32", [None])
initializer = tf.uniform_unit_scaling_initializer(1.0)
# 
Wi = tf.get_variable('Wi', [n_input, n_hidden], initializer = initializer)
Wh = tf.get_variable('Wh', [n_hidden, n_hidden], initializer = initializer)
bh = tf.get_variable('bh', [n_hidden], initializer=tf.constant_initializer(0.01))
Wy = tf.get_variable('Wy', [n_hidden, n_classes], initializer = initializer)
by = tf.get_variable('by', [n_classes], initializer=tf.constant_initializer(0.01))
tf.get_variable_scope().reuse_variables()
outputs = tensor_array_ops.TensorArray(dtype=tf.float32, size=0, dynamic_size=True,\
     tensor_array_name="outputs", clear_after_read=False)

In [ ]:
def DRNN(x, batch_len):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps)
    # Required shape: 'n_steps' tensors list of shape (batch_size)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    
    #Get largest len in batch
    largest_len = tf.reduce_max(batch_len)
    #TensorArray

    iterator = tf.constant(0, dtype=tf.int32)
    state_0 = tf.constant(0.0, shape=[batch_size, n_hidden])
    #Max time step is not fixed as largest_len
    def rnn_cell(rnn_input, state):
        n_state = tf.tanh(tf.matmul(rnn_input, Wi) + tf.matmul(state, Wh) + bh)
        output = tf.matmul(n_state, Wy) + by
        return n_state, output
    def cond(iterator, outputs, inputs, state):
        return tf.less(iterator, largest_len)
    def body(iterator, outputs, inputs, state):
        rnn_input = tf.slice(inputs, [iterator, 0, 0], [1, batch_size, n_input])
        rnn_input = tf.reshape(rnn_input, [batch_size, n_input])
        n_state, output = rnn_cell(rnn_input, state)
        #Write output
        outputs = outputs.write(iterator, output)
        return iterator+1, outputs, inputs, n_state
    _, rnn_outputs, _, last_state = tf.while_loop(cond, body, [iterator, outputs, x, state_0])
    rnn_outputs = rnn_outputs.pack()
    rnn_outputs.set_shape([None, batch_size, n_classes]) 
    # Permuting batch_size and n_steps
    rnn_outputs = tf.transpose(rnn_outputs,perm=[1, 0, 2])
    # Linear activation, using rnn inner loop last output
    return rnn_outputs

pred = DRNN(x, len_info)
final_outputs = tf.reshape(pred, [-1, n_classes])

# Define loss and optimizer
cost = tf.nn.seq2seq.sequence_loss([final_outputs], [tf.reshape(y, [-1])], [tf.reshape(mask, [-1])])

#Calculate gradients
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


outputs.close()

# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = \
            input_label[step%epoch_per_step], y_label[step%epoch_per_step]
        batch_input_len , batch_mask = \
            input_length[step%epoch_per_step], input_mask[step%epoch_per_step]
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        batch_y = batch_y.reshape((batch_size, n_steps))
        # Run optimization op (backprop)
        _,  loss = sess.run([optimizer, cost],\
                                feed_dict={x: batch_x, y: batch_y ,\
                                len_info: batch_input_len, mask: batch_mask})

        if step % display_step == 0:
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss)
        step += 1
    print "Optimization Finished!"
